# PHYS2923 SSP Project


### Aim
Initially the aim of this code is to simulate ~100 particles in a 1nm*4 square box. The goal is to study how certain thermodynamic variables such as temperature, pressure, entropy behave and whether we can make sense of them on this small scale. 

### Defining parameters

#### Box
* The box will consist of 4 walls which are $10^{-9}$m in length
* One wall will be an adjustable wall which can be moved in similar to a piston
* The wall will essentially consist of **hard**, but infinitesimally small particles which elastically collide with the particles inside the box
* The walls will be used to measure pressure

#### Particles
* Each particle will be circular, and have a radius of the Bohr radius $a_0 = 5.29177210903\cdot10^{-11}$m
* Each particle will have a mass equal to that of Hydrogen 1.00784 AMU
* Each particle will be described using the following three vectors:

* Position $ \vec x = (x, y)$

* Velocity $ \vec v = (v_x, v_y)$

* Acceleration $ \vec a = (a_x, a_y)$

### Forces acting on the particles
#### Elastic repulsive force between particles
For two particles $A$ and $B$ if:

$$distance(\vec x_a, \vec x_b) \le 2a_0$$

At this point, our particles will have an elastic collision in $2-D$ whereby the conservation laws of momentum and kinetic energy. For each particle, its velocity must be split into the component along the direction of the collision and one component perpendicular to the collision. Since momentum (and hence velocity) is only imparted along the direction of the collision, the component perpendicular remains unchanged. The velocities along the line of collision can then be changed according the the equations of a $1-D$ collision 

###### Begin with a 1D collision
To begin with a $1-D$ collision, first let us consider the relevant laws:

Momentum: $\vec p = m \vec v$
<br>
Kinetic energy: $KE = \frac{1}{2} m \vec v^2$

Conservation of **momentum**: 
$$\vec p_a + \vec p_b = \vec p_a' + \vec p_b'$$

Conservation of **kinetic energy**:
$$\frac{1}{2} m_a \vec v_a^2 + \frac{1}{2} m_b \vec v_b^2 = \frac{1}{2} m_a \vec v_a'^2 + \frac{1}{2} m_b \vec v_b'^2$$

This leaves us with a final result if we work through the algebra:

$$ v_a' = \frac{v_a (m_a - m_b) + 2 m_b v_b}{m_a + m_b}$$
$$ v_b' = \frac{v_b (m_b - m_a) + 2 m_a v_a}{m_a + m_b}$$

We can use this in our 2-D collision system

##### Elastic Collisions in 2D

1. Firstly we need to find the unit normal and unit tangent vectors with respect to the line of the collision. 

The normal vector
$$ \vec n = ( x_b - x_a, y_b - y_a)$$

Next we find the unit vector of $ \vec n$, denoted $\hat n$
$$ \hat n = \frac{\vec n}{|\vec n|} = \frac{\vec n}{\sqrt{n_x^2 + n_y^2}}$$

To generate the unit tangent vector $\hat t$:

$$ \hat t = (-\hat n_y, \hat n_x)$$

2. Now we need to project the velocity components of our particles onto the unit normal and unit tangent vectors using the dot product:

$$v_{an} = \hat n \cdot \vec v_a$$
$$v_{at} = \hat t \cdot \vec v_a$$

$$v_{bn} = \hat n \cdot \vec v_b$$
$$v_{at} = \hat n \cdot \vec v_t$$

3. Find the new tangential velocities after the collision. This is quite simple, they are unaffected.

$$v'_{at} = v_{at}$$
$$v'_{bt} = v_{bt}$$

4. Now we have to find the new normal velocities. Here we use the one dimensional collision formulas
$$ v_{an}' = \frac{v_{an} (m_a - m_b) + 2 m_b v_{bn}}{m_a + m_b}$$
$$ v_{bn}'= \frac{v_{bn} (m_b - m_a) + 2 m_a v_{an}}{m_a + m_b}$$

5. Now we convert the scalar normal and tangential normal velocities back into vectors. We do this by multiplying the unit normal vector by the scalar normal velocity (cf. tangential)

$$\vec v_{an}' = v_{an}' \cdot \hat n$$
$$\vec v_{at}' = v_{at}' \cdot \hat t$$

$$\vec v_{bn}' = v_{bn}' \cdot \hat n$$
$$\vec v_{bt}' = v_{bt}' \cdot \hat t$$

6. Then we create our new velocity vectors by by adding the normal and tangential components of each particle:

$$ \vec v_a' = \vec v_{an}' + \vec v_{at}'$$
$$ \vec v_b' = \vec v_{bn}' + \vec v_{bt}'$$

7. The last step is to resolve the  these vectors back into $(v_x, v_y)$

We do this by by projecting the velocity vectors onto the unit vectors in the $x$ and $y$ directions:
$$ \vec v_a' = (\hat i \cdot \vec v_a', \hat j \cdot \vec v_a') = (v_{ax}', v_{ay}')$$
$$ \vec v_b' = (\hat i \cdot \vec v_b', \hat j \cdot \vec v_b') = (v_{bx}', v_{by}')$$

Given that our Velocity Verlet algorithm uses acceleration to apply changes to velocity, we cannot simply instantaneously change velocity. We need to convert the changes in velocity from this collision to equivalent acceleration in order for the algorithm to work. To do this:

$$ \vec a_a' = \frac{(\vec v_a' - \vec v_a)}{\Delta t}$$
$$ \vec a_b' = \frac{(\vec v_b' - \vec v_b)}{\Delta t}$$

##### Collisions with the wall
Since the particle is to be confined to a box, when the particle hits a wall we need to define the behaviour. The way to do this is to check the distance between the position coordinates of the particle and each of the walls. The way to do this is to check the distance between the center coordinates $(x_0,y_0)$ and a line defined by two points $(P_1, P_2)$ see [Wikipedia](https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line):

$$distance(P_1, P_2, (x_0, y_0)) = \frac{|(x_2 - x_1)(y_1 - y_0) - (x_1 - x_0)(y_2 - y_1)|}{\sqrt{(x_2 -x_1)^2 +(y_2 -y_1)^2}}$$


There are a few cases which can occur here. 

1. The particle is closer than or equal to its radius from the **one** wall. In the case that it is a vertical wall, the horizontal velocity vector of the particle changes sign. If it is a horizontal wall, the vertical velocity vector changes sign
2. In the case the particle is closer to 2 wall (corners) both velocity vectors are changed. 

###### Representing our Box
Consider we have 4 points which represent our walls and one point representing the center of our particle. We have 2 limits, an $x_{lim}$ and a $y_{lim}$. Thus we have four points representing our box:

* Left-Bottom Corner: $(0,0)$
* Left-Top Corner: $(0, y_{lim})$
* Right-Bottom Corner: $(x_{lim}, 0)$
* Right-Top Corner: $(x_{lim}, y_{lim})$

Thus there are two horizontal lines and two vertical lines defined by their end points:

* Vertical Line 1: $((0,0),(0, y_{lim}))$
* Vertical Line 2: $((x_{lim}, 0),(x_{lim}, y_{lim}))$
* Horizontal Line 1: $((0,0),(x_{lim}, 0)$
* Horizontal Line 2: $((0,y_{lim}),(x_{lim}, y_{lim}))$

###### Collisions with one wall
1. Vertical collisions

$$If$$
$$distance((0,0),(0, y_{lim}), (x_0, y_0)) \le a_0 $$

$$or $$

$$distance((x_{lim}, 0),(x_{lim}, y_{lim}), (x_0, y_0)) \le a_0$$

$$then$$

$$ \vec v' = (-v_x, v_y)$$

Given then that we need to think about changes in velocity in terms of acceleration, we have:

$$ \vec a' = \frac{(v_x'-v_x,v_y'-v_y)}{\Delta t} = \frac{(-v_x-v_x,0)}{\Delta t} =\frac{(-2v_x,0)}{\Delta t}$$

2. Horizontal collisions

$$If$$
$$distance((0,0),(x_{lim}, 0), (x_0, y_0)) \le a_0 $$

$$or $$
$$distance((0,y_{lim}),(x_{lim}, y_{lim})) \le a_0$$

$$then$$

$$ \vec v' = (v_x, -v_y)$$

Given then that we need to think about changes in velocity in terms of acceleration, we have:

$$ \vec a' = \frac{(v_x'-v_x,v_y'-v_y)}{\Delta t} = \frac{(0,-v_y-v_y)}{\Delta t} =\frac{(0,-2v_y)}{\Delta t}$$

###### Collisions with 2 walls
In the case that the distance between a particle and any two walls is less than or equal to $a_0$ then both the $x$ and $y$ velocity components reverse. Could this be more complex?

$$ \vec v' = (-v_x, -v_y)$$

Given then that we need to think about changes in velocity in terms of acceleration, we have:

$$ \vec a' = \frac{(v_x'-v_x,v_y'-v_y)}{\Delta t} = \frac{(-v_x-v_x,-v_y-v_y)}{\Delta t} =\frac{(-2v_x,-2v_y)}{\Delta t}$$

#### Attractive potential between particles
TBC

### Velocity Verlet Algorithm
Suppose we have a system with N particles. As we have discussed, each particle $i$ has position, velocity and acceleration vectors at a particular time $t$ denoted by:
* Position $ \vec x_i(t) = (x_i, y_i)$

* Velocity $ \vec v_i(t) = (v_{ix}, v_{iy})$

* Acceleration $ \vec a_i(t) = (a_{ix}, a_{iy})$

Importantly, we assume our Acceleration $ \vec a_i(t)$ depends only on position not on velocity. 

First we begin with a particle with a known set of position, velocity and acceleration vectors at a time $t$:
$ \vec x_i(t) = (x_0, y_0)$

$ \vec v_i(t) = (v_{x0}, v_{y0})$

$ \vec a_i(t) = (a_{x0}, a_{x0})$

Now we calculate our new position and velocity after time $\Delta t$:

$$ \vec x_i(t + \Delta t) = x_i(t) + v_i(t)\Delta t + \frac{1}{2}a_i(t) \Delta t^2$$

$$ \vec v_i(t + \Delta t) = v_i(t) + \frac{\vec a(t) + \vec a(t + \Delta t)}{2} \Delta t$$

We can simplify the algorithm to the following:

1. Calculate $ \vec x_i(t + \Delta t) = x_i(t) + v_i(t)\Delta t + \frac{1}{2}a_i(t) \Delta t^2$
2. Calculate $ \vec a_i(t) $ using our particle interactions
3. Calculate $ \vec v_i(t + \Delta t) = v_i(t) + \frac{\vec a(t) + \vec a(t + \Delta t)}{2} \Delta t$